In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/amiyaguchi/wikipedia-retention


In [12]:
! ls data/processed

2007-1-enwiki-projection-user.csv	 all_user_features.csv
2007-1-enwiki-projection-user-roles.csv  base_features_reg.csv
2007Q1-user-network-v1.csv		 community_norm_features.csv
2007Q1-user-network-v2.csv		 enwiki-meta-compact
all_article_features.csv		 kcore-2007-1.csv


# Spark Preprocessing

In [4]:
from src.data.snap_import_user_projection import UnimodalUserProjection
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()

input_path = "data/processed/enwiki-meta-compact"
model = UnimodalUserProjection(spark).extract(input_path).transform()

In [11]:
import shutil
import glob
import os

def write_csv(df, name, overwrite=False):
    interim_path = "data/interim/{}".format(name)
    processed_file = "data/processed/{}.csv".format(name)
    if os.path.exists(processed_file) and not overwrite:
        print("file already exists")
        return
    df.repartition(1).write.csv(interim_path, header=True, mode="overwrite")
    interim_file = glob.glob("{}/*.csv".format(interim_path))[0]
    shutil.copy(interim_file, processed_file)
    shutil.rmtree(interim_path)
    print("wrote file to {}".format(processed_file))

In [5]:
spark.table("bipartite").cache()
spark.table("bipartite").count()

53599976

In [15]:
projection = spark.sql("""
select
    t1.user_id as e1,
    t2.user_id as e2,
    t1.article_id
from bipartite t1
join bipartite t2
on t1.article_id = t2.article_id
where t1.user_id < t2.user_id
    and t1.edit_date = t2.edit_date
""")

projection.count()

19362468

In [16]:
write_csv(projection, "user-network-intermediate-edgelist")

file already exists


In [18]:
block_list = spark.sql("""
with block_list as (
    select
        article_id,
        edit_date,
        collect_set(user_id) as user_set
    from bipartite
    group by 1, 2
)
select 
    article_id,
    edit_date,
    size(user_set) as n_users,
    user_set
from block_list
""")

block_list.count()

44869198

In [21]:
(
    block_list
    .repartition(1)
    .write.parquet("data/processed/user-network-intermediate-block-userset")
)